In [1]:
%defaultDatasource jdbc:h2:mem:db

# Analisando o Uso de Medicamentos
* Esta tarefa analisará os dados conforme três abordagens: análise do comportamento estatístico; análise de correlação entre pares; análise de correlação de variáveis.
* A partir dos arquivos CSV que estão no diretórios /data/nhanes considere as seguintes tabelas SQL.

In [2]:
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

CREATE TABLE Medicamento (
  Codigo VARCHAR(6) NOT NULL,
  NomeMedicamento VARCHAR(100) NOT NULL,
  PRIMARY KEY(Codigo)
) AS SELECT
    Codigo,
    NomeMedicamento
FROM CSVREAD('../../../data/nhanes/medications-drug.csv');

CREATE TABLE UsoMedicamento (
  IdPessoa VARCHAR(5) NOT NULL,
  CodMedicamento VARCHAR(6) NOT NULL,
  DiasUso INTEGER,
  FOREIGN KEY(IdPessoa)
    REFERENCES Pessoa(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(CodMedicamento)
    REFERENCES Medicamento(Codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    IdPessoa,
    CodMedicamento,
    DiasUso
FROM CSVREAD('../../../data/nhanes/medications-use.csv');

In [ ]:
SELECT * FROM Pessoa;
SELECT * FROM Medicamento;
SELECT * FROM UsoMedicamento;

# I. Análise de comportamento estatístico

## 1) Contabilizando o uso de medicamentos

* Liste o nome de todos os medicamentos e a quantidade de pessoas que usa cada um deles.

In [56]:
SELECT Med.NomeMedicamento, COUNT(*) Quantidade
    FROM Medicamento Med, UsoMedicamento UsoMed
    WHERE Med.Codigo = UsoMed.CodMedicamento
    GROUP BY UsoMed.CodMedicamento;

![](1.PNG)

## 2) Medicamento mais usado com VIEW

* Informe o código do medicamento mais usado: fazendo uso de VIEW.

In [8]:
DROP VIEW IF EXISTS Usuarios;

CREATE VIEW Usuarios AS
SELECT CodMedicamento, Count(*) AS Valor
    FROM UsoMedicamento
    GROUP BY CodMedicamento;
    SELECT Usuarios.CodMedicamento
        FROM Usuarios
        ORDER BY Valor DESC
        LIMIT 1;

d00732

In [57]:
DROP VIEW IF EXISTS MedicamentoMaisUsado;

CREATE VIEW MedicamentoMaisUsado AS
SELECT MAX(Valor) Uso
    FROM Usuarios;
    SELECT qtd.CodMedicamento, num.Uso
        FROM Usuarios qtd, MedicamentoMaisUsado num
        WHERE qtd.Valor = num.Uso;

![](2.PNG)

## 3) Medicamento mais usado sem VIEW

* Informe o código do medicamento mais usado: sem fazer uso de VIEW.

In [58]:
SELECT CodMedicamento, COUNT(*) Uso
    FROM UsoMedicamento
    GROUP BY CodMedicamento
    ORDER BY Uso DESC
    LIMIT 1;

![](3.PNG)

## 4) Nome do medicamento mais usado
* Informe o nome do medicamento mais usado (uso de VIEW é opcional).

In [59]:
SELECT Med.NomeMedicamento, Count(*) Uso
    FROM Medicamento Med, UsoMedicamento UsoMed
    WHERE Med.Codigo = UsoMed.CodMedicamento
    GROUP BY UsoMed.CodMedicamento
    ORDER BY Uso DESC
    LIMIT 1;

![](4.PNG)

## 5) Contabilizando quanto as pessoas usam de medicamento

* Informe o número médio de uso de medicamento por pessoa.

In [32]:
DROP VIEW IF EXISTS MedicamentoQuantidade;

CREATE VIEW MedicamentoQuantidade AS
SELECT UsoMedicamento.IdPessoa, COUNT(*) Valor
    FROM UsoMedicamento
    GROUP BY UsoMedicamento.IdPessoa;
    SELECT AVG(MedicamentoQuantidade.Valor)
        FROM MedicamentoQuantidade

3

## 6) Pessoas que usam mais do que a média

* Liste o id das pessoas que usam mais medicamentos do que a média.

In [60]:
DROP VIEW IF EXISTS MediaMedicamento;

CREATE VIEW MediaMedicamento AS
SELECT AVG(MedicamentoQuantidade.Valor) Media
    FROM MedicamentoQuantidade;
    SELECT UsoMedicamento.IdPessoa, COUNT(*) QuantidadeUso
        FROM UsoMedicamento, MediaMedicamento
        GROUP BY IdPessoa
        HAVING QuantidadeUso > MediaMedicamento.Media

![](6.PNG)

## 7) Análise do uso de medicamento

* Considere que um médico quer fazer responder a seguinte questão: pessoas tendem a usar mais medicamentos conforme ficam mais velhas?
* Escreva uma query que ajude o médico a realizar esta análise.

In [61]:
SELECT Pessoa.Idade, AVG(MedicamentoQuantidade.Valor)
    FROM MedicamentoQuantidade, Pessoa
    WHERE MedicamentoQuantidade.IdPessoa = Pessoa.Id
    GROUP BY Pessoa.Idade

![](7.PNG)

# II. Análise de correlação em pares
## Estudo de ação cruzada entre medicamentos

* Um pesquisador quer estudar o efeito cruzado de usar dois medicamentos simultaneamente. 
* Para isso ele precisará da seguinte sequência de queries.

## 8) Medicamentos tomados em conjunto
* Construa uma VIEW com duas colunas: medicamentoA, medicamentoB
* Nessa VIEW, é criada uma tupla (medicamentoA, medicamentoB) toda vez que uma pessoa toma o medicamento A e também o medicamento B.
* Haverá repetição de tuplas, já que mais de uma pessoa pode tomar o mesmo par de medicamentos.
* Cuidado para não duplicar a mesma informação, ou seja, considerando que uma pessoa tomou o medicamentoX e o medicamentoY, cuidado para não criar duas tuplas: (medicamentoX, medicamentoY) e (medicamentoY, medicamentoX).

In [ ]:
DROP VIEW MedicamentoB IF EXISTS;
CREATE VIEW MedicamentoB AS
SELECT IdPessoa PesB, CodMedicamento MedB
    FROM UsoMedicamento;

In [63]:
DROP VIEW Conjunto IF EXISTS;
CREATE VIEW Conjunto AS
SELECT MedUm.CodMedicamento A, MedDois.MedB B
    FROM UsoMedicamento MedUm, MedicamentoB MedDois
    WHERE MedUm.IdPessoa = MedDois.PesB AND MedUm.CodMedicamento > MedDois.MedB;

SELECT * FROM Conjunto;

![](8.PNG)

## 9) Total de medicamento tomados em conjunto
* Apresente o total de pessoas que toma cada par de medicamentos.

In [64]:
SELECT A, B, COUNT(*) QuantidadeConjunta
    FROM Conjunto
    GROUP BY A, B;

![](9.PNG)

# III. Análise de correlação entre variáveis
* Consiste na análise da seguinte questão: como a alteração de uma variável afeta a outra.
## Matriz de análise
* Considere a matriz abaixo.

In [ ]:
CREATE TABLE Matriz (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  IdadeAte60 BOOLEAN,
  MaisUmAnoMedicamento BOOLEAN,
  MedicamentosAcimaMedia BOOLEAN,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade,
    FALSE, FALSE, FALSE
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

SELECT * FROM Matriz;

## 10) Atualização simples da matriz
* Utilize o comando UPDATE para atualizar a coluna IdadeAte60 da tabela colocando verdadeiro para as pessoas que têm até 60 anos de idade.

In [65]:
UPDATE Matriz
SET IdadeAte60 = TRUE
WHERE Idade <=60;

SELECT * FROM Matriz;

![](10.PNG)

## 11) Atualização vinculada a uma segunda tabela
* Utilize o comando UPDATE para atualizar a coluna MaisUmAnoMedicamento, colocando verdadeiro para aquelas pessoas que usam pelo menos um medicamento há mais de um ano.

In [66]:
SELECT DISTINCT IdPessoa
             FROM UsoMedicamento
             WHERE DiasUso > 365;

UPDATE Matriz
SET MaisUmAnoMedicamento = TRUE
WHERE Id IN (SELECT DISTINCT IdPessoa
             FROM UsoMedicamento
             WHERE DiasUso > 365);
             
SELECT * FROM Matriz;

![](11.PNG)

## 12) Atualização baseada em uma consulta complexa
* Utilize o comando UPDATE para atualizar a coluna MedicamentosAcimaMedia, colocando verdadeiro para aquelas pessoas que usam um número de medicamentos acima da média.

In [67]:
UPDATE Matriz
SET MedicamentosAcimaMedia = TRUE
WHERE Id IN (
    SELECT IdPessoa
    FROM UsoMedicamento
    GROUP BY IdPessoa
    HAVING COUNT(*) > (
        SELECT AVG(NumeroMedicamentos) MediaMedicamentos
        FROM (SELECT COUNT(*) Numeromedicamentos
              FROM UsoMedicamento
              GROUP BY IdPessoa)));
              
SELECT * FROM Matriz WHERE MedicamentosAcimaMedia = TRUE;

![](12.PNG)

## 13) Que análise interessante pode ser feita?
* A partir da Matriz criada, proponha uma análise interessante.

### 13a) Descreva a seguir (em Markdown) qual a análise que você propõe.

Observando a tabela, podemos ver que pessoas que usam um número de medicamentos acima da média também marcam TRUE para o uso de pelo menos um medicamento há mais de um ano.

Fazendo uma consulta, no 13b), é possível observar que dos 1486 resultados totais de pessoa que usam um número de medicamentos acima da média, apenas 72 não fazem o uso de pelo menos um medicamento há mais de um ano.

### 13b) Escreva uma consulta em SQL que fornece informações para a análise proposta.

In [68]:
SELECT * FROM Matriz WHERE MedicamentosAcimaMedia = TRUE AND MaisUmAnoMedicamento = FALSE;

![](13.PNG)